In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [21]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode

Conectando ao Neo4j 

In [22]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [26]:
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query1).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository[405:500].reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:a_arte_de_acumular_na_gestacao...,http://www.teses.usp.br/teses/disponiveis/12/1...
1,tag:stardog:api:identidade_e_territorialidade_...,http://www.teses.usp.br/teses/disponiveis/8/81...
2,tag:stardog:api:o_fim_da_escravidao_sob_a_otic...,https://www.teses.usp.br/teses/disponiveis/106...
3,tag:stardog:api:sweating_system_e_trabalho_esc...,https://www.teses.usp.br/teses/disponiveis/100...
4,tag:stardog:api:os_trabalhadores_bolivianos_em...,http://www.teses.usp.br/teses/disponiveis/2/21...
...,...,...
90,tag:stardog:api:cidadania_socioambiental_da_mu...,https://www.teses.usp.br/teses/disponiveis/2/2...
91,tag:stardog:api:quando_falha_o_controle_crimes...,http://www.teses.usp.br/teses/disponiveis/8/81...
92,tag:stardog:api:na_roda_das_donas_do_samba_um_...,https://www.teses.usp.br/teses/disponiveis/27/...
93,tag:stardog:api:guias_e_orixas_narrativas_de_e...,http://www.teses.usp.br/teses/disponiveis/8/81...


Criando as utils functions

In [27]:

# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, verify=True).text#, verify=False) 
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):
            path = doc['href']
            prefix_uni = 'https://www.teses.usp.br'
            link = prefix_uni + path

            return link
    
    return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):
    # NOTE the stream=True parameter below
    r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese 3.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  
      
        # Removendo aspas
        if "'" in page_text:
            page_text = page_text.replace("'", "")
        if '"' in page_text:
            page_text = page_text.replace('"', '')


        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        
        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [28]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link}")
        download_file(pdf_link, thesis_uri)
        
        # Extraindo o texto e entidades
        tese_path = 'tese 3.pdf'
        excerpt = extratc_text(tese_path)

        excerpts = []
        persons = []
        gpes = []

        # Salvando as triplas em CSV
        for n in excerpt.keys():

            excerpt_uri = thesis_repository['Thesis'][i] + '_p_' + str(n)
            excerpt_text = excerpt[n]['text']

            excerpt_page = n
            excerpt_lang = excerpt[n]['lang']
            
            excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, str(excerpt_page), excerpt_lang])
                
            #Criando as relações com as entidades
            for person in excerpt[n]['persons']:
                persons.append([excerpt_uri, 'tag:stardog:api:' + person])
            
            for gpe in excerpt[n]['gpes']:
                gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
        
        # Salvando os dados em CSV no diretório de importação do Neo4j
        pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts3.csv', index=False, header=False)
        pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons3.csv', index=False, header=False)
        pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes3.csv', index=False, header=False)
        
        #Deletando o arquivo PDF após o processamento
        os.remove('tese 3.pdf')

        print(f"Loading triplas: {thesis_uri}")

        # Importanto os dados para o Neo4j
        #Excerpts
        query = """
        LOAD CSV FROM 'file:///excerpts3.csv'
        AS row
        MATCH  (t:Thesis{uri: row[0]})
        MERGE  (e:Exerpt{
                uri: row[1],
                text: row[2],
                page: row[3],
                lang: row[4]
                })-[r:BFO_0000050]-(t);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #Persons
        query = """
        LOAD CSV FROM 'file:///persons3.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Person{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #GPEs
        query = """
        LOAD CSV FROM 'file:///gpes3.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Place{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        # Removendo os arquivos CSV após a importação
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts3.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons3.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes3.csv')

        print(f"Total time: {pd.Timestamp.now() - start_time}")
        print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
        # Atualizando a hora parcial
        parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()

0 - Downloading and extracting: a_arte_de_acumular_na_gestacao_da_economia_cafeeira_formas_de_enriquecimento_no_vale_do_paraiba_paulista_durante_o_seculo_xix from https://www.teses.usp.br/teses/disponiveis/12/12138/tde-26112008-155949/publico/TESE1.pdf
Loading triplas: a_arte_de_acumular_na_gestacao_da_economia_cafeeira_formas_de_enriquecimento_no_vale_do_paraiba_paulista_durante_o_seculo_xix
Total time: 0 days 00:12:48.173435
Partial time: 0 days 00:12:48.173435
1 - Downloading and extracting: identidade_e_territorialidade_os_quilombos_e_a_educacao_escolar_no_vale_do_ribeira from https://www.teses.usp.br/teses/disponiveis/8/8135/tde-01122015-175909/publico/2006_LisangelaKatiDoNascimento.pdf
Loading triplas: identidade_e_territorialidade_os_quilombos_e_a_educacao_escolar_no_vale_do_ribeira
Total time: 0 days 00:25:43.870073
Partial time: 0 days 00:12:55.696638
2 - Downloading and extracting: o_fim_da_escravidao_sob_a_otica_da_historia_da_energia_um_estudo_com_foco_da_era_maua_brasil_18

Transaction failed and will be retried in 0.8239329913001403s (ForsetiClient[transactionId=18431, clientId=2] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=18421, clientId=1]} on NODE_RELATIONSHIP_GROUP_DELETE(128249) because holders of that lock are waiting for ForsetiClient[transactionId=18431, clientId=2].
 Wait list:ExclusiveLock[
Client[18421] waits for [ForsetiClient[transactionId=18431, clientId=2]]])


Total time: 0 days 00:31:21.352120
Partial time: 0 days 00:05:37.483055
3 - Downloading and extracting: sweating_system_e_trabalho_escravo_contemporaneo_um_estudo_sobre_a_repeticao_de_padroes from https://www.teses.usp.br/teses/disponiveis/100/100133/tde-24012024-165955/publico/defesa_final_2023.pdf
Loading triplas: sweating_system_e_trabalho_escravo_contemporaneo_um_estudo_sobre_a_repeticao_de_padroes
Total time: 0 days 00:43:22.584223
Partial time: 0 days 00:12:01.231095
4 - Downloading and extracting: os_trabalhadores_bolivianos_em_sao_paulo_uma_abordagem_juridica from https://www.teses.usp.br/teses/disponiveis/2/2138/tde-03092012-145034/publico/DISSERTACAO_INTEGRAL_Gabrielle_Louise_Soares_Timoteo.pdf
Loading triplas: os_trabalhadores_bolivianos_em_sao_paulo_uma_abordagem_juridica
Total time: 0 days 00:47:31.044298
Partial time: 0 days 00:04:08.460075
5 - Downloading and extracting: narrativas_de_capoeiras_por_capoeiristas_na_moenda_viva_da_territorializacao_do_estado_brasileiro fro

KeyboardInterrupt: 